In [1]:
#dependencies
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup 
from splinter import Browser
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

In [2]:
geo_path = "../../data/state_data/geo/ppp_geo.csv" #data as of 11/21/21
ppp_geo= pd.read_csv(geo_path)
ppp_geo.shape[0]

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (69) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


968557

In [3]:
ppp_geonull=ppp_geo.loc[ppp_geo["Lat"].isnull()] #457467
ppp_geonull.head()

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
369,369,369,5048318601,3/20/21,1084,PPS,DELTA INDUSTRIAL SERVICES INC.,1229 RICHARDSON HWY,DELTA JUNCTION,AK,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"1229 RICHARDSON HWY, DELTA JUNCTION, AK",NaN,NaN,NaN
428,428,428,5085327008,4/5/20,1084,PPP,"DELTA INDUSTRIAL SERVICES, INC.",1229 Richardson Hwy,DELTA JUNCTION,AK,...,999 Other,5511 Management of Companies and Enterprises,5511.0,ALASKA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1229 Richardson Hwy, DELTA JUNCTION, AK",NaN,NaN,NaN
1865,1865,1865,4637628408,2/6/21,1084,PPS,MCKINLEY FIRE PROTECTION LLC,1706 Ship Ave,Anchorage,AK,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 Ship Ave, Anchorage, AK",NaN,NaN,NaN
2274,2274,2274,5337318401,2/8/21,1084,PPS,THE ALASKA CENTRE FOR CREATIVE RENEWAL LLC,46877 STILLPOINT TRL,HALIBUT COVE,AK,...,999 Other,7211 Traveler Accommodation,7211.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"46877 STILLPOINT TRL, HALIBUT COVE, AK",NaN,NaN,NaN
2379,2379,2379,9891427104,4/15/20,1084,PPP,MCKINLEY FIRE PROTECTION,1706 SHIP AVENUE,ANCHORAGE,AK,...,999 Other,2382 Building Equipment Contractors,2382.0,ALASKA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"1706 SHIP AVENUE, ANCHORAGE, AK",NaN,NaN,NaN


In [4]:
ppp_geonull["BorrowerState"].value_counts()

TX    76183
NY    74112
MI    29806
WA    23754
FL    22314
CO    19741
MN    19730
WI    17354
LA    13668
OR    13456
UT     9295
IA     8295
NE     5804
HI     4980
NM     4485
ID     4416
ND     3222
MT     3144
MO     3120
PR     2952
SD     2695
OK     2535
IN     2348
WY     2139
SC     2045
MS     1589
NV     1569
KS     1485
GU      443
RI      414
VI      290
MP       82
NC       11
AL        8
PA        8
AK        5
AZ        4
NJ        3
MA        3
TN        3
AR        2
ME        2
IL        2
GA        1
WV        1
CT        1
OH        1
Name: BorrowerState, dtype: int64

In [5]:
ppp_togeoFL = ppp_geonull.loc[ppp_geonull["BorrowerState"] == "FL"]
ppp_togeoFL #60040

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
205913,205913,205913,1442247205,4/15/20,455,PPP,COLE SCOTT & KISSANE PA,9150 S Dadeland Blvd. Ste 1400,MIAMI,FL,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL",NaN,NaN,NaN
205916,205916,205916,3130317300,4/29/20,491,PPP,"SNI COMPANIES, INC.",7751 Belfort Parkway,SAINT JOHNS,FL,...,999 Other,"5416 Management, Scientific, and Technical Con...",5416.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"7751 Belfort Parkway, SAINT JOHNS, FL",NaN,NaN,NaN
205919,205919,205919,4658808010,6/26/20,455,PPP,"SILVER AIRWAYS, LLC",1100 LEE WAGENER BLVD STE 201,FORT LAUDERDALE,FL,...,999 Other,4881 Support Activities for Air Transportation,4881.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"1100 LEE WAGENER BLVD STE 201, FORT LAUDERDALE...",NaN,NaN,NaN
205928,205928,205928,6717997003,4/7/20,491,PPP,GRAND PERFORMER INC.,4901 Vineland Road Suite 650,ORLANDO,FL,...,999 Other,5613 Employment Services,5613.0,NORTH FLORIDA DISTRICT OFFICE,e $5-10 million,500 or more,"4901 Vineland Road Suite 650, ORLANDO, FL",NaN,NaN,NaN
205931,205931,205931,7574857210,4/28/20,455,PPP,BOIES SCHILLER FLEXNER LLP,2200 Corporate Blvd NW Suite 400,BOCA RATON,FL,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,e $5-10 million,250 to 499,"2200 Corporate Blvd NW Suite 400, BOCA RATON, FL",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265946,265946,265946,9940717704,5/1/20,455,PPP,"M.R. FRIEDMAN & G.A. FRIEDMAN, P.A.",2600 Douglas Rd,Coral Gables,FL,...,999 Other,5411 Legal Services,5411.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"2600 Douglas Rd, Coral Gables, FL",NaN,NaN,NaN
265947,265947,265947,9950428308,1/31/21,455,PPS,DIGESTIVE & LIVER DISEASE CONSULTANTS OF SOUTH...,7369 Sheridan St,Hollywood,FL,...,999 Other,6211 Offices of Physicians,6211.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"7369 Sheridan St, Hollywood, FL",NaN,NaN,NaN
265949,265949,265949,9977848305,1/31/21,455,PPS,BCG FACILITY SERVICES LLC,8608 Little Rd,New Port Richey,FL,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"8608 Little Rd, New Port Richey, FL",NaN,NaN,NaN
265950,265950,265950,9988227001,4/9/20,455,PPP,"SOUTH TAMPA SWIM SCHOOL, LLC",5016 N COOLIDGE AVE,TAMPA,FL,...,999 Other,6116 Other Schools and Instruction,6116.0,SOUTH FLORIDA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"5016 N COOLIDGE AVE, TAMPA, FL",NaN,NaN,NaN


In [6]:
#process to identify and code for unique addresses 
cols = ["LoanNumber","full_add"]
ppp_togeoFL_add = ppp_togeoFL[cols]
ppp_togeoFL_add

,LoanNumber,full_add
205913,1442247205,"9150 S Dadeland Blvd. Ste 1400, MIAMI, FL"
205916,3130317300,"7751 Belfort Parkway, SAINT JOHNS, FL"
205919,4658808010,"1100 LEE WAGENER BLVD STE 201, FORT LAUDERDALE..."
205928,6717997003,"4901 Vineland Road Suite 650, ORLANDO, FL"
205931,7574857210,"2200 Corporate Blvd NW Suite 400, BOCA RATON, FL"
...,...,...
265946,9940717704,"2600 Douglas Rd, Coral Gables, FL"
265947,9950428308,"7369 Sheridan St, Hollywood, FL"
265949,9977848305,"8608 Little Rd, New Port Richey, FL"
265950,9988227001,"5016 N COOLIDGE AVE, TAMPA, FL"


In [7]:
add = ppp_togeoFL["full_add"].astype(str).to_list()

In [ ]:
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/onelineaddress?form'
driver.visit(url)

not_found = []
lat = []
long = []
geo_id = []

i=0
for i in tqdm(range(len(add))):
        driver.find_by_id('_address_id').clear()
        driver.find_by_id('_address_id').fill(add[i])
        driver.find_by_id('proceed').click()
        
        soup = BeautifulSoup(driver.html, 'html.parser')
        results = soup.find_all('div', id="pl_gov_census_geo_geocoder_domain_AddressResult")
        
        for result in results:
            result_text = result.text
            result_lat = re.findall('[^-](\d{2}[.]+\d+)', result_text)
            result_long = re.findall('[-]\d{2}[.]+\d+', result_text) #WEST 3-digit Longitude
            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('12')] #FL
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{add[i]} is {result_lat}, {result_long}, {fips_elements}")
                lat.append(result_lat[0])
                long.append(result_long[0])
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                not_found.append(i)
                lat.append(None)  
                long.append(None)
                geo_id.append(None)  
                print(f"{add[i]} is Not Found")
            except (TimeoutException):
                driver.navigate.refresh()
    
        sleep()
        i=i+1

In [9]:
print(len(add))
print(len(lat))
print(len(long))
print(len(geo_id))

22314
22313
22312
22312


<b>Add remaining values (2801) to result_df(5461)

In [24]:
pd.set_option("display.max_rows", None)
lat_df = pd.DataFrame({"Lat":lat})
lat_df_drop = lat_df
#lat_df_drop.loc[lat_df_drop["Lat"] == "12.18", "Lat"] ="None"
lat_df_drop.loc[lat_df_drop["Lat"] == "20.18", "Lat"] ="26.08465"
#lat_df_drop.loc[lat_df_drop["Lat"] == "37.5", "Lat"] ="None"
lat_df_drop.loc[lat_df_drop["Lat"] == "11.13", "Lat"] ="None"

In [ ]:
lat_df_sort = lat_df.sort_values(by="Lat")
lat_df_sort

In [25]:
lat_df_drop = lat_df_drop.drop(index=[12216], axis=1)
lat_df_drop = lat_df_drop.reset_index()
lat_df_drop = lat_df_drop.drop(["index"], axis=1)
lat_list = lat_df_drop["Lat"].to_list()

In [40]:
add_df = pd.DataFrame({"add": add})
add_df_list = add_df["add"].to_list()

In [38]:
result_df = pd.DataFrame({ "Lat":lat_list, "Long":long, "FIPS_z": geo_id})
result_df.loc[12216, "Lat"] = "25.783415"
result_df

line = pd.DataFrame({"Lat": None, "Long": None, "FIPS_z": None}, index=[16659.5])
df2 = result_df.append(line, ignore_index=False)
df2 = df2.sort_index().reset_index(drop=True)
line2 = pd.DataFrame({"Lat": None, "Long": None, "FIPS_z": None}, index=[20187.5])
df3 = df2.append(line2, ignore_index=False)
df3 = df3.sort_index().reset_index(drop=True)

In [ ]:
df3["full_add"] = add_df_list
df3

In [42]:
result_df = df3

In [ ]:
result_df['FIPS_z'] = result_df['FIPS_z'].str[0]
result_df 

In [44]:
#mapping coordinates to NaN values in main 
m = dict(zip(result_df.full_add,result_df.Lat))
ppp_togeoFL.Lat = ppp_togeoFL.Lat.fillna(ppp_togeoFL.full_add.map(m))
n = dict(zip(result_df.full_add,result_df.Long))
ppp_togeoFL.Long = ppp_togeoFL.Long.fillna(ppp_togeoFL.full_add.map(n))
o = dict(zip(result_df.full_add,result_df.FIPS_z))
ppp_togeoFL.FIPS_z = ppp_togeoFL.FIPS_z.fillna(ppp_togeoFL.full_add.map(o))

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [45]:
cols = ["LoanNumber", "FIPS_z", "Lat", "Long"]
ppp_togeoFL_parsed = ppp_togeoFL[cols]
ppp_togeoFL_parsed.head()

,LoanNumber,FIPS_z,Lat,Long
205913,1442247205,None,None,None
205916,3130317300,None,None,None
205919,4658808010,None,None,None
205928,6717997003,None,None,None
205931,7574857210,None,None,None


In [ ]:
ppp_geo = ppp_geo.merge(ppp_togeoFL_parsed, on="LoanNumber", how="left")
ppp_geo.head()

In [47]:
ppp_geo["Lat"] = ppp_geo["Lat_y"].fillna(ppp_geo["Lat_x"])
ppp_geo["Long"] = ppp_geo["Long_y"].fillna(ppp_geo["Long_x"])
ppp_geo["FIPS_z"] = ppp_geo["FIPS_z_y"].fillna(ppp_geo["FIPS_z_x"])
ppp_geo = ppp_geo.drop(["Lat_x", "Long_x", "Lat_y", "Long_y", "FIPS_z_y", "FIPS_z_x"], axis=1)
ppp_geo.head()

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,0,9547507704,5/1/20,464,PPP,"SUMTER COATINGS, INC.",2410 Highway 15 South,Sumter,SC,...,325 Chemicals,"3255 Paint, Coating, and Adhesive Manufacturing",3255.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"2410 Highway 15 South, Sumter, SC",33.8537,-80.3699,NaN
1,1,1,9777677704,5/1/20,464,PPP,"PLEASANT PLACES, INC.",7684 Southrail Road,North Charleston,SC,...,999 Other,5617 Services to Buildings and Dwellings,5617.0,SOUTH CAROLINA DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"7684 Southrail Road, North Charleston, SC",32.9423,-80.0662,NaN
2,2,2,5791407702,5/1/20,1013,PPP,BOYER CHILDREN'S CLINIC,1850 BOYER AVE E,SEATTLE,WA,...,NaN,NaN,NaN,SEATTLE DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"1850 BOYER AVE E, SEATTLE, WA",47.6361,-122.303,NaN
3,3,3,6223567700,5/1/20,920,PPP,KIRTLEY CONSTRUCTION INC,1661 MARTIN RANCH RD,SAN BERNARDINO,CA,...,999 Other,2361 Residential Building Construction,2361.0,SANTA ANA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"1661 MARTIN RANCH RD, SAN BERNARDINO, CA",34.2247,-117.373,NaN
4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.5218,-82.9137,NaN


In [48]:
ppp_geo.to_csv("data/state_data/geo/ppp_geo.csv")

In [49]:
ppp_geo_fl = ppp_geo.loc[ppp_geo["BorrowerState"] =="FL"]
ppp_geo_fl.shape[0]

60041

In [54]:
null_check = ppp_geo_fl.loc[ppp_geo_fl["Lat"].isnull()]
null_check.shape[0]

10045